### Exploratory Data Analysis with Federated BigQuery - Part 1

In this notebook we will query two silos of data:
  - **1 silo in GCS**: Reference table: 'next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1'
  - **1 silo in CloudSQL**: The queries goes against an external connection.
  
The query syntax is the following:

- The first silo in GCS is setup as an external permanent table so the query syntax is the same as native tables in BigQuery.
- The second silo in CloudSQL is setup as an external connection. Therefore, the query syntax need the following:

<pre><code>
SELECT * FROM EXTERNAL_QUERY("projects/next-bigquery-omni-poc-es-gcp/locations/europe-west2/connections/postgre_connection", "SELECT * FROM INFORMATION_SCHEMA.TABLES;");
</code></pre>

#### import bigquery python api

In [3]:
from google.cloud import bigquery

client = bigquery.Client()

#### import visualization libraries

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
%matplotlib inline

#### 00 - Get a sample of data

In [9]:
sql = """
SELECT * FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` LIMIT 10
"""
df_result = client.query(sql).to_dataframe()

In [11]:
df_result.head()

,int64_field_0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
1,4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
2,5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
3,11,12,15737173,Andrews,497,Spain,Male,24,3,0.00,2,1,0,76390.01,0
4,14,15,15600882,Scott,635,Spain,Female,35,7,0.00,2,1,1,65951.65,0


Warning!!! The limit does not decrease the number of rows or bytes to process

#### 01- Compute the number of distinct columns and rows

In [12]:
sql = """
SELECT  
    count(distinct column_name) 
,  (select  count(*) from  `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`)
FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'sp_silo1'
"""
df_result = client.query(sql).to_dataframe()

In [13]:
df_result.head()

,f0_,f1_
0,15,2477


#### 02- Describe some basic statistics

In [14]:
sql_describe_salary = """
SELECT 
  MIN(EstimatedSalary) AS min_salary
  , MAX(EstimatedSalary) As max_salary
  , COUNT(EstimatedSalary) AS num_salary
  , AVG(EstimatedSalary) AS avg_salary
  , SUM(EstimatedSalary) AS total_salary
  , STDDEV(EstimatedSalary) AS stddev_salary
  , VARIANCE(EstimatedSalary) AS variance_salary
FROM
 FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1`
 WHERE table_name = 'sp_silo1'
"""

*Automated procedure in SQL equivalent to describe method in Pandas*

In [15]:
sql_describe = """
DECLARE columns ARRAY<STRING>;
DECLARE query1, query2, query3, query4, query5, query6, query7 STRING;
SET columns = (
  WITH all_columns AS (
    SELECT column_name
    FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'sp_silo1' 
        and  data_type IN ('INT64','FLOAT64')
  )
  SELECT ARRAY_AGG((column_name) ) AS columns
  FROM all_columns
);

SET query1 = (select STRING_AGG('(select stddev( '||x||')  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`) '||x ) AS string_agg from unnest(columns) x );
SET query2 = (select STRING_AGG('(select avg( '||x||')  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`) '||x ) AS string_agg from unnest(columns) x );
SET query3 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.5) over()  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query4 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.25) over()  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query5 = (select STRING_AGG('(select PERCENTILE_CONT( '||x||', 0.75) over()  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` limit 1) '||x ) AS string_agg from unnest(columns) x );
SET query6 = (select STRING_AGG('(select max( '||x||')  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`) '||x ) AS string_agg from unnest(columns) x );
SET query7 = (select STRING_AGG('(select min( '||x||')  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`) '||x ) AS string_agg from unnest(columns) x );

EXECUTE IMMEDIATE (
"SELECT 'stddev' ,"|| query1 || " UNION ALL " ||
"SELECT 'mean'   ,"|| query2 || " UNION ALL " ||
"SELECT 'median' ,"|| query3 || " UNION ALL " ||
"SELECT '0.25' ,"|| query4 || " UNION ALL " ||
"SELECT '0.75' ,"|| query5 || " UNION ALL " ||
"SELECT 'max' ,"|| query6 || " UNION ALL " ||
"SELECT 'min' ,"|| query7
)
;
"""

df_describe = client.query(sql_describe).to_dataframe()

In [16]:
df_describe.head()

,f0_,int64_field_0,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,min,1.000000,2.000000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,417.410000,0.000000
1,max,9992.000000,9993.000000,1.581569e+07,850.000000,88.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000
2,mean,4949.667743,4950.667743,1.569192e+07,651.333872,38.890997,5.032297,61818.147763,1.539362,0.694792,0.529673,99440.572281,0.166734
3,stddev,2881.454458,2881.454458,7.176554e+04,94.365051,10.446119,2.856660,64235.555208,0.564646,0.460588,0.499220,57103.678091,0.372813
4,0.25,2355.000000,2356.000000,1.562954e+07,587.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50267.690000,0.000000


#### 03- Get outliers of Estimated Salary

In [21]:
sql_outliers = """
DECLARE lower, upper, mean FLOAT64;
SET mean = (select avg( EstimatedSalary)  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`);
SET lower = mean - 2 * (select stddev( EstimatedSalary)  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`);
SET upper = mean + 2 * (select stddev( EstimatedSalary)  from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`);
EXECUTE IMMEDIATE (
"SELECT * from `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` WHERE EstimatedSalary >"|| upper ||" OR EstimatedSalary < " || lower
)
"""

df_outliers = client.query(sql_outliers).to_dataframe()

In [22]:
df_outliers.head()

,int64_field_0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


#### 04- Remove NULL rows

In [24]:
sql_drop_null = """
DECLARE columns ARRAY<STRING>;
DECLARE query STRING DEFAULT '';
DECLARE i INT64 DEFAULT 0;

SET columns = (
  WITH all_columns AS (
    SELECT column_name
    FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = 'sp_silo1' 
        -- and  data_type IN ('INT64','FLOAT64')
  )
  SELECT ARRAY_AGG((column_name) ) AS columns
  FROM all_columns
);

LOOP
    SET i = i + 1;

    IF i > ARRAY_LENGTH(columns) THEN 
        LEAVE;
    END IF;
 
    SET query = ' DELETE FROM  `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1` WHERE ' || columns[ORDINAL(i)] || ' is null '  ;
    EXECUTE IMMEDIATE (
        query
    );

END LOOP;
"""


df_drop_null = client.query(sql_drop_null).to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/next-bigquery-omni-poc-es-gcp/queries/f44bdb61-12e6-4cf2-923d-9cf9229c820d?maxResults=0&location=europe-west2&prettyPrint=false: Invalid value: DML statements are only supported over tables that have data stored in BigQuery. Unsupported table: next-bigquery-omni-poc-es-gcp:churn_data_silo_1.sp_silo1 at [1:2]

(job ID: f44bdb61-12e6-4cf2-923d-9cf9229c820d)

                                                        -----Query Job SQL Follows-----                                                        

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:DECLARE columns ARRAY<STRING>;
   3:DECLARE query STRING DEFAULT '';
   4:DECLARE i INT64 DEFAULT 0;
   5:
   6:SET columns = (
   7:  WITH all_columns AS (
   8:    SELECT column_name
   9:    FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.INFORMATION_SCHEMA.COLUMNS`


In [ ]:
df_drop_null.head()

#### 05- Correlations 

In [26]:
sql_corr = """
SELECT corr(CreditScore,Balance) FROM `next-bigquery-omni-poc-es-gcp.churn_data_silo_1.sp_silo1`
"""

df_corr = client.query(sql_corr).to_dataframe()

In [27]:
df_corr

,f0_
0,-0.0074


*** --> PoC unfinished ***

### Conclusions

* **01**- BiQuery is not well suited for a exploratory data analysis. 
  - The most common workflow seems to be load the data into a DataFrame: https://www.cloudskillsboost.google/focuses/1161?parent=catalog
* **02**- It is not possible to perform DML operations. Although the documenation is not clear in this aspect, it has been tested in the point 04.
* **03**- It is not possible to perform JOIN operations.